In [100]:
import pandas as pd
import pandas as pd
import geopandas as gpd
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool
from bokeh.palettes import Viridis256, Inferno256, Magma256
from bokeh.palettes import BrBG, PiYG, RdGy, RdYlGn, YlGnBu
from bokeh.models import LinearColorMapper, ColorBar
#import streamlit as st
#from streamlit_folium import st_folium
#import folium
#from leaflet import Map, GeoJSON

In [101]:
def format_df(filename):
    df = pd.read_csv(filename)
    df['harvest'] = pd.to_datetime(df['harvest'], format='%Y/%m/%d')
    df['planting'] = pd.to_datetime(df['planting'], format='%Y/%m/%d')
    df['date'] = df['harvest'].dt.strftime("%Y")
    df = df.rename(columns={'cname': 'Region'})
    df['gwad'] =df['gwad']/1000
    #df = df.rename(columns={df.columns[1]: "Region"})
    #df['Region'] = df['Region'].str.upper()
    return df

In [102]:
unimodal=format_df('rheas/tanzania_tamsat_25km_districts_unimod_dssatTable_2021_2023_100kg.csv')
bimodal = format_df('rheas/tanzania_tamsat_25km_districts_bimod_dssatTable_2021_2023_100kg.csv')

In [103]:
# Unimodal season:  October--August (LR)
lrStart = [9, 10, 11, 12, 1,2, 3, 4]
lrEnd = [2, 3, 4, 5, 6, 7, 8]
def LR_metrics(lr, lrStart, lrEnd, season):
    lr.loc[(pd.to_datetime(lr['planting']).dt.month.isin(lrStart)) & (pd.to_datetime(lr['harvest']).dt.month.isin(lrEnd)), 'Season'] = season
    lr = lr.groupby(['Region', 'date', 'Season'], as_index=False).agg({'gwad': 'mean'})

    return lr

rh_lr = LR_metrics(unimodal, lrStart, lrEnd, "LR")
print(rh_lr.head())

     Region  date Season      gwad
0      Bahi  2022     LR  3.783650
1      Bahi  2023     LR  3.123037
2   Buhigwe  2022     LR  3.725214
3   Buhigwe  2023     LR  2.691574
4  Chamwino  2022     LR  3.743050


In [104]:
rh_lr23=rh_lr[rh_lr["date"] == '2023'] 
print(rh_lr23.head())

     Region  date Season      gwad
1      Bahi  2023     LR  3.123037
3   Buhigwe  2023     LR  2.691574
5  Chamwino  2023     LR  3.158562
7    Chemba  2023     LR  3.077475
9    Chunya  2023     LR  3.254950


In [105]:
# Bimodal season B (MAM): January--August
mamStart = [1, 2, 3, 4]
mamEnd = [5, 6, 7, 8]
def MAM_metrics(mam, mamStart, mamEnd, season):
    mam.loc[(pd.to_datetime(mam['planting']).dt.month.isin(mamStart)) & (pd.to_datetime(mam['harvest']).dt.month.isin(mamEnd)), 'Season'] = season
    mam = mam.groupby(['Region', 'date', 'Season'], as_index=False).agg({'gwad': 'mean'})

    return mam

rh_mam = MAM_metrics(bimodal, mamStart, mamEnd, "MAM")
print(rh_mam.head())

         Region  date Season      gwad
0        Arusha  2022    MAM  3.560500
1        Arusha  2023    MAM  2.902175
2  Arusha Urban  2022    MAM  2.847075
3  Arusha Urban  2023    MAM  2.751550
4        Babati  2022    MAM  3.567550


In [106]:
rh_mam23=rh_mam[rh_mam["date"] == '2023'] 
print(rh_mam23.head())

         Region  date Season      gwad
1        Arusha  2023    MAM  2.902175
3  Arusha Urban  2023    MAM  2.751550
5        Babati  2023    MAM  3.223975
7  Babati Urban  2023    MAM  2.961275
9      Bagamoyo  2023    MAM  3.369000


In [107]:
unimodal_shp = gpd.read_file("/Users/gamoyo/Documents/GitHub/yieldprediction/rheas/gadm36_TZA_2_LongRain.shp")
bimodal_shp = gpd.read_file("/Users/gamoyo/Documents/GitHub/yieldprediction/rheas/gadm36_TZA_2_Bimodal.shp")
unimodal_shp.rename(columns={'NAME_2':'Region'}, inplace=True)
bimodal_shp.rename(columns={'NAME_2':'Region'}, inplace=True)
country_shapefile = gpd.read_file('/Users/gamoyo/Documents/GitHub/yieldprediction/rheas/gadm36_TZA_0.shp')
#shapefile["NewDist20"] = shapefile.index
#print(shapefile)
LR23 = pd.merge(unimodal_shp, rh_lr23, on="Region")
MAM23 = pd.merge(bimodal_shp, rh_mam23, on="Region")

In [108]:
geo_sourceLR23 = GeoJSONDataSource(geojson=LR23.to_json())
geo_sourceMAM23 = GeoJSONDataSource(geojson=MAM23.to_json())
country_geo_source = GeoJSONDataSource(geojson=country_shapefile.to_json())

In [109]:
color_mapper = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
p1 = figure(title='Long Rains Yield (Mt/Ha) - 2023',
           plot_width=650, plot_height=600)

p1.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p1.patches('xs','ys', source=geo_sourceLR23,
          fill_color={'field': 'gwad', 'transform': color_mapper},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('District', '@Region'), ('gwad', '@gwad')])
p1.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p1.add_layout(color_bar, 'right')

In [110]:
color_mapper1 = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
p2 = figure(title='MAM Yield (Mt/Ha) - 2023',
           plot_width=650, plot_height=600)

p2.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p2.patches('xs','ys', source=geo_sourceMAM23,
          fill_color={'field': 'gwad', 'transform': color_mapper1},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('District', '@Region'), ('gwad', '@gwad')])
p2.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper1, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p2.add_layout(color_bar, 'right')

In [111]:
#from bokeh.layouts import column
#layout = column(p1, p2)
#show(layout)

In [112]:
p=gridplot([[p1, p2]])
show(p)

In [116]:
from bokeh.io import save
# specify the output file name and location
save(p, filename='yield_map.html', title='Yield Data Map')

'/Users/gamoyo/Documents/GitHub/yieldprediction/yield_map.html'

In [114]:
#from bokeh.io import save

# specify the output file name and location
#save(p, filename='yield_map.html', title='Yield Data Map')